# [Create fold with Livedoor Dataset | Kaggle](https://www.kaggle.com/code/co21cen/create-fold-with-livedoor-dataset)

In [1]:
%cd ~

/root


## コーパスのダウンロード

In [2]:
!ls work/data/corpus/scripts/download_livedoor_news_corpus.sh

work/data/corpus/scripts/download_livedoor_news_corpus.sh


In [ ]:
import subprocess

cmd = "sh work/data/corpus/scripts/download_livedoor_news_corpus.sh"
subprocess.run(cmd, shell=True)

## データの整形

In [4]:
import glob
import os

import pandas as pd
from sklearn import model_selection, preprocessing

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


In [1]:
from work.config.path.livedoor_news_corpus_paths import LivedoorNewsCorpusPaths

In [5]:
text_dir = LivedoorNewsCorpusPaths.LIVEDOOR_NEWS_CORPUS_TEXT_DIR
files_folders = [name for name in os.listdir(text_dir)]

categories = [name for name in os.listdir(text_dir) if os.path.isdir(text_dir + name)]

print("category length:", len(categories))
print("categories     :", categories)

category length: 9
categories     : ['dokujo-tsushin', 'it-life-hack', 'kaden-channel', 'livedoor-homme', 'movie-enter', 'peachy', 'smax', 'sports-watch', 'topic-news']


In [6]:
def extract_main_txt(file_name):
    with open(file_name) as text_file:
        # extract only text
        text = text_file.readlines()[3:]

        text = [sentence.strip() for sentence in text]
        text = list(filter(lambda line: line != "", text))
        text = "".join(text)
        text = text.translate(
            str.maketrans({"\n": "", "\t": "", "\r": "", "\u3000": ""})
        )
        return text

In [7]:
class LivedoorNewsCorpusColumns:
    TEXT = "text"
    CATEGORY = "category"
    CATEGORY_LABEL = "category_label"

In [8]:
list_text = []
list_label = []

for cat in categories:
    text_files = glob.glob(os.path.join(text_dir, cat, "*.txt"))

    body = [extract_main_txt(text_file) for text_file in text_files]
    label = [cat] * len(body)

    list_text.extend(body)
    list_label.extend(label)

In [9]:
df = pd.DataFrame(
    {
        LivedoorNewsCorpusColumns.TEXT: list_text,
        LivedoorNewsCorpusColumns.CATEGORY: list_label,
    }
)

In [10]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(df[LivedoorNewsCorpusColumns.CATEGORY].values)
df[LivedoorNewsCorpusColumns.CATEGORY_LABEL] = y
print("data shape :", df.shape)
df.head()

data shape : (7376, 3)


,text,category,category_label
0,自由に転載・引用が可能です。このディレクトリの記事ファイル内容の提供元：独女通信http:/...,dokujo-tsushin,0
1,もうすぐジューン・ブライドと呼ばれる６月。独女の中には自分の式はまだなのに呼ばれてばかり……...,dokujo-tsushin,0
2,携帯電話が普及する以前、恋人への連絡ツールは一般電話が普通だった。恋人と別れたら、手帳に書か...,dokujo-tsushin,0
3,「男性はやっぱり、女性の“すっぴん”が大好きなんですかね」と不満そうに話すのは、出版関係で働...,dokujo-tsushin,0
4,ヒップの加齢による変化は「たわむ→下がる→内に流れる」、バストは「そげる→たわむ→外に流れる...,dokujo-tsushin,0


In [11]:
df[LivedoorNewsCorpusColumns.CATEGORY].unique()

array(['dokujo-tsushin', 'it-life-hack', 'kaden-channel',
       'livedoor-homme', 'movie-enter', 'peachy', 'smax', 'sports-watch',
       'topic-news'], dtype=object)

In [12]:
df[LivedoorNewsCorpusColumns.CATEGORY_LABEL].unique()

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

## 保存

In [13]:
LivedoorNewsCorpusPaths.LIVEDOOR_NEWS_CORPUS_RAW_PATH

'work/data/corpus/livedoor_news/livedoor_news_corpus_ldcc-20140209.csv'

In [14]:
filepath = LivedoorNewsCorpusPaths.LIVEDOOR_NEWS_CORPUS_RAW_PATH
df.to_csv(filepath, index=False)